In [14]:
import datasets
import pandas as pd

hf_data = datasets.load_dataset("pufanyi/cassava-leaf-disease-classification")

In [15]:
whole_data = hf_data["train"]
df = pd.DataFrame(whole_data)
df.head()

,image_id,image,label
0,1000015157.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,0
1,1000201771.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,3
2,100042118.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1
3,1000723321.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1
4,1000812911.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,3


In [16]:
labels = df["label"].unique()
labels

array([0, 3, 1, 2, 4])

In [17]:
data = {}
for label in labels:
    data[label] = df[df["label"] == label]

In [18]:
for label, df in data.items():
    print(f"Label {label}: {len(df)} samples")

Label 0: 1087 samples
Label 3: 13158 samples
Label 1: 2189 samples
Label 2: 2386 samples
Label 4: 2577 samples


In [19]:
val = []
train = []
for label in labels:
    val.append(data[label].sample(100))
    train.append(data[label].drop(val[-1].index))

In [20]:
val_df = pd.concat(val)
train_df = pd.concat(train)

In [21]:
len(val_df), len(train_df)

(500, 20897)

In [26]:
def gen(df: pd.DataFrame):
    for i, row in df.iterrows():
        yield row.to_dict()


final_data = {
    "train": datasets.Dataset.from_generator(
        gen, gen_kwargs={"df": train_df}, features=hf_data["train"].features
    ),
    "validation": datasets.Dataset.from_generator(
        gen, gen_kwargs={"df": val_df}, features=hf_data["train"].features
    ),
    "test": hf_data["test"],
}

Generating train split: 20897 examples [00:39, 523.51 examples/s]
Generating train split: 500 examples [00:00, 500.38 examples/s]


In [27]:
final_data

{'train': Dataset({
     features: ['image_id', 'image', 'label'],
     num_rows: 20897
 }),
 'validation': Dataset({
     features: ['image_id', 'image', 'label'],
     num_rows: 500
 }),
 'test': Dataset({
     features: ['image_id', 'image', 'label'],
     num_rows: 1
 })}

In [28]:
final_data = datasets.DatasetDict(final_data)

In [29]:
final_data.push_to_hub("pufanyi/cassava-leaf-disease-classification")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.34s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/pufanyi/cassava-leaf-disease-classification/commit/eaa8e90768df4bdaa011f78858fe83886d8c3dee', commit_message='Upload dataset', commit_description='', oid='eaa8e90768df4bdaa011f78858fe83886d8c3dee', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/pufanyi/cassava-leaf-disease-classification', endpoint='https://huggingface.co', repo_type='dataset', repo_id='pufanyi/cassava-leaf-disease-classification'), pr_revision=None, pr_num=None)